# CS779 Final Evaluation pipeline Task-1

# CODALAB submission 4 for scibert+bi-lstm

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Training Code


In [ ]:
!git clone https://github.com/ncg-task/training-data.git "/content/train"

Cloning into '/content/train'...
remote: Enumerating objects: 3083, done.
remote: Counting objects: 100% (3083/3083), done.
remote: Compressing objects: 100% (2728/2728), done.
remote: Total 6864 (delta 567), reused 2504 (delta 279), pack-reused 3781
Receiving objects: 100% (6864/6864), 157.36 MiB | 39.88 MiB/s, done.
Resolving deltas: 100% (660/660), done.
Checking out files: 100% (3286/3286), done.


In [ ]:
!git clone https://github.com/ncg-task/trial-data.git "/content/valid"

Cloning into '/content/valid'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (1636/1636), done.
remote: Compressing objects: 100% (1438/1438), done.
remote: Total 1636 (delta 580), reused 1224 (delta 170), pack-reused 0
Receiving objects: 100% (1636/1636), 27.88 MiB | 39.00 MiB/s, done.
Resolving deltas: 100% (580/580), done.


In [ ]:
####Loading Training Data
#input_dir = "/content/drive/My Drive/Train v2/"
input_dir = "/content/train/"
import os
import torch
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
#list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#                   "relation-classification","text-classification"]
list_of_folders = ["query_wellformedness", "passage_re-ranking", "part-of-speech_tagging", 
         "sentence_compression", "sentiment_analysis", "temporal_information_extraction", 
         "phrase_grounding", "text_generation", "text-to-speech_synthesis", 
         "smile_recognition", "topic_models", "question_generation", 
         "relation_extraction", "paraphrase_generation", "question_similarity", 
         "question_answering", "sentence_classification", "prosody_prediction", 
         "semantic_role_labeling", "text_summarization", "semantic_parsing",
         "sarcasm_detection", "natural_language_inference", "negation_scope_resolution"]
input_stanza_list = []
input_stanza_len = []
input_sent_num_list = []
file_name_list = []
for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(sentence_num_list)
    file_name_list.append(fls + '/' + str(i))
  print("completed",fls, count)

completed query_wellformedness 1
completed passage_re-ranking 2
completed part-of-speech_tagging 8
completed sentence_compression 4
completed sentiment_analysis 52
completed temporal_information_extraction 2
completed phrase_grounding 1
completed text_generation 6
completed text-to-speech_synthesis 3
completed smile_recognition 1
completed topic_models 1
completed question_generation 2
completed relation_extraction 14
completed paraphrase_generation 2
completed question_similarity 1
completed question_answering 6
completed sentence_classification 3
completed prosody_prediction 1
completed semantic_role_labeling 5
completed text_summarization 15
completed semantic_parsing 3
completed sarcasm_detection 2
completed natural_language_inference 101
completed negation_scope_resolution 1


In [ ]:
##### Validation Dataset Loading
#val_input_dir = "/content/drive/My Drive/Dev_v2/"
val_input_dir = "/content/valid/"
val_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
         "relation-classification", "text-classification"]
val_input_stanza_list = []
val_input_sent_num_list = []
val_file_name_list = []
val_input_stanza_len = []
for fls in val_list_of_folders:
  count=0
  for i in os.listdir(val_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(val_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        val_input_stanza_len.append(len(stanza_lines_list))
        val_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        val_input_sent_num_list.append(sentence_num_list)
    val_file_name_list.append(fls + '/' + str(i))
  print("completed",fls,count)

completed machine-translation 10
completed named-entity-recognition 10
completed question-answering 10
completed relation-classification 10
completed text-classification 10


In [ ]:
##########Check Flag for Train and Validation set
print("Train no. of examples for each stanza,sentences and entities")
print(len(input_stanza_list),len(input_sent_num_list),len(input_stanza_len),len(file_name_list))
input_sent_num_list = [[int(s) for s in sublist] for sublist in input_sent_num_list] # convert sentence list string to integer
input_sent_num_list = [list(set(x)) for x in input_sent_num_list]
print("print one example to show all files")
print(input_sent_num_list[0],len(input_stanza_list[0]),file_name_list[0],max(input_stanza_len))
t_sentence = 0
for t in range(len(input_sent_num_list)):
  t_sentence += len(input_sent_num_list[t])
print(f"Total number of sentences in training set {t_sentence} and examples {len(input_sent_num_list)}")
##########Check validation
print("Valid no. of examples for each stanza,sentences and entities")
print(len(val_input_stanza_list),len(val_input_sent_num_list),len(val_input_stanza_len),len(val_file_name_list))
val_input_sent_num_list = [[int(s) for s in sublist] for sublist in val_input_sent_num_list] # convert sentence list string to integer
val_input_sent_num_list = [list(set(x)) for x in val_input_sent_num_list]
print("print one example to show all files")
print(val_input_sent_num_list[0],len(val_input_stanza_list[0]),val_file_name_list[0],max(val_input_stanza_len))

val_t_sentence = 0
for i in range(len(val_input_sent_num_list)):
  val_t_sentence += len(val_input_sent_num_list[i])
print(f"Total number of sentences in valid set {val_t_sentence} and examples {len(val_input_sent_num_list)}")
input_stanza_list[0][1]

Train no. of examples for each stanza,sentences and entities
237 237 237 237
print one example to show all files
[2, 71, 72, 19, 20, 21, 24, 58, 59, 60] 96 query_wellformedness/0 513
Total number of sentences in training set 5064 and examples 237
Valid no. of examples for each stanza,sentences and entities
50 50 50 50
print one example to show all files
[32, 33, 130, 131, 2, 4, 129, 9, 25, 26, 27, 28, 29, 30, 31] 143 machine-translation/5 387
Total number of sentences in valid set 1029 and examples 50


'identifying well - formed natural language questions'

In [ ]:
###Ground truth label formation for Train and Validation
###############Train
multihot_input_sent = []
for i in range(len(input_stanza_list)):
  temp =[0]*input_stanza_len[i]
  for j in range(len(input_sent_num_list[i])):
    t1 = input_sent_num_list[i][j] -1
    temp[t1] = 1
  multihot_input_sent.append(temp)
#################valid
val_multihot_input_sent = []
for i in range(len(val_input_stanza_list)):
  temp =[0]*val_input_stanza_len[i]
  for j in range(len(val_input_sent_num_list[i])):
    t1 = val_input_sent_num_list[i][j] -1
    temp[t1] = 1
  val_multihot_input_sent.append(temp)

In [ ]:
##Function for flattening 2d list
from collections import Iterable
def flatten(lis):
  for item in lis:
    if isinstance(item, Iterable) and not isinstance(item, str):
      for x in flatten(item):
        yield x
    else:        
      yield item

In [ ]:
####### Flatten Train and Validation nested list for sentence and their labels
from collections import Counter

train_sentences = list(flatten(input_stanza_list))
train_label = list(flatten(multihot_input_sent))
valid_sentences = list(flatten(val_input_stanza_list))
valid_label = list(flatten(val_multihot_input_sent))
print("train examples",len(train_sentences),len(train_label))
print("valid examples",len(valid_sentences),len(valid_label))
train_tuple = list(set((zip(train_sentences,train_label))))
train_in_sentences = []
train_sent_label = []
for stan,lab in train_tuple:
  if len(stan) >4:
    train_in_sentences.append(stan)
    train_sent_label.append(lab)
#train_in_sentences, train_sent_label = zip(*train_tuple)
valid_tuple = list(set((zip(valid_sentences,valid_label))))
valid_in_sentences = []
valid_sent_label = []
for stan,lab in valid_tuple:
  if len(stan) >4:
    valid_in_sentences.append(stan)
    valid_sent_label.append(lab)
#valid_in_sentences, valid_sent_label = zip(*valid_tuple)
print("train examples",len(train_in_sentences),len(train_sent_label))
print("valid examples",len(valid_in_sentences),len(valid_sent_label))
print("train count", Counter(train_sent_label))

In [ ]:
###Oversampling of minority class
train_tuple = list(set((zip(train_in_sentences,train_sent_label))))
train_sum_sent = 5 * [stan for stan,label in train_tuple if label==1]
train_nonsum_sent = [stan for stan,label in train_tuple if label==0]
print(len(train_sum_sent), len(train_nonsum_sent))
train_in_sentences = train_sum_sent + train_nonsum_sent
train_sent_label = len(train_sum_sent)*[1] + len(train_nonsum_sent)*[0]
print("Oversampled train examples",len(train_in_sentences),len(train_sent_label))

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 1.8MB 21.4MB/s 
     |████████████████████████████████| 2.9MB 52.5MB/s 
     |████████████████████████████████| 890kB 61.3MB/s 


In [ ]:
########### Pre-processing complete for train and valid. 
########### Start model building
import numpy as np
from sklearn import metrics
import transformers
import torch
import random
from transformers import AutoTokenizer, AutoModel
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
##### Parameters for the model
EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')			
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32

In [ ]:
#### Dataset Loader
class Triage(Dataset):
    def __init__(self, data_in,data_out, tokenizer, max_len):
        self.len = len(data_in)
        self.data = data_in
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label = data_out
        
    def __getitem__(self, index):
        # title = str(self.data.TITLE[index])
        # title = " ".join(title.split())
        title = self.data[index]
        inputs = self.tokenizer.encode_plus(title,
            None,add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
            return_length = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']
        lengths = inputs['length']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'lengths': torch.tensor(lengths, dtype=torch.long),
            'targets': torch.tensor(self.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
      return self.len

In [ ]:
#### Train and Validation Loader 
training_set = Triage(train_in_sentences,train_sent_label, tokenizer, MAX_LEN)
train_params = {'batch_size': TRAIN_BATCH_SIZE,'shuffle': True,'num_workers': 0}
training_loader = DataLoader(training_set, **train_params)
valid_set = Triage(valid_in_sentences,valid_sent_label, tokenizer, MAX_LEN)
valid_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0}
valid_loader = DataLoader(valid_set, **valid_params)

In [ ]:
# Create a SciBERT + Bi-LSTM model for binary classification
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        #self.lstm_hidden_size = lstm_hidden_size
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800, 400)
        self.l4 = torch.nn.Linear(400,100) # check with layer with 30
        self.l5 = torch.nn.Linear(100,2)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        #bert_hidden_layer = bert_hidden_layer.permute(1, 0, 2)   #permute rotates the tensor. if tensor.shape = 3,4,5  tensor.permute(1,0,2), then tensor,shape= 4,3,5  (batch_size, sequence_length, hidden_size)
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        #output_3 = self.l2(output_2)
        output_4 = self.l4(output_2)
        #output_4 = torch.nn.ReLU()(output_4)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)

In [ ]:
#### Optimizer and Loss function
optimizer = torch.optim.AdamW(params = model.parameters(), lr=LEARNING_RATE)
c_weights = torch.tensor([list(Counter(train_sent_label).values())[0], list(Counter(train_sent_label).values())[1]], dtype=torch.float32).to(device)
c_weights = 1.0/(c_weights/c_weights.sum())
c_weights = c_weights/c_weights.sum()
loss_function = torch.nn.CrossEntropyLoss(weight = c_weights)
print("class weights",c_weights)
def calcuate_accu(big_idx, targets):
  n_correct = torch.sum((big_idx==targets) * (targets==1)).item()
  return n_correct
def calculate_labels(targets):
  return torch.sum(targets==1).item()
valid_loss_list = []

In [ ]:
#### Function for training one epoch
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_predict = 0
    n_ground = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        lengths = data['lengths'].squeeze(1)#.to(device, dtype = torch.long)
        #print("print length shape", lengths.shape, ids.shape, token_type_ids.shape)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask,token_type_ids, lengths)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        n_predict +=calculate_labels(big_idx)
        n_ground +=calculate_labels(targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
            #print(f"Training Accuracy per 1000 steps: {accu_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    #print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch {epoch}: {epoch_accu} and Loss: {epoch_loss}")
    P = n_correct/n_predict if n_predict else 0.0
    R = n_correct/n_ground if n_ground else 0.0
    F1 = (2*P*R)/(P+R) if P+R else 0.0
    print(f"Training P and R and F1 Epoch {epoch}: {P},{R}, {F1}")
    return

In [ ]:
#### Function for validation one epoch
def valid(epoch,model, valid_loader):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_correct = 0
    n_predict = 0
    n_ground = 0
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            lengths = data['lengths'].squeeze(1)#.to(device, dtype = torch.long)
            #print("print length shape", lengths.shape, ids.shape, token_type_ids.shape)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask,token_type_ids, lengths)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            
            n_predict +=calculate_labels(big_idx)
            n_ground +=calculate_labels(targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            # if _%100==0:
            #     loss_step = tr_loss/nb_tr_steps
            #     accu_step = (n_correct*100)/nb_tr_examples
            #     print(f"Validation Loss per 100 steps: {loss_step}")
            #     print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    #print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch {epoch}: {epoch_accu} and Loss: {epoch_loss}")
    valid_loss_list.append(epoch_loss)
    P = n_correct/n_predict if n_predict else 0.0
    R = n_correct/n_ground if n_ground else 0.0
    F1 = (2*P*R)/(P+R) if P+R else 0.0
    print(f"Valiadation  P and R and F1 Epoch {epoch}: {P},{R}, {F1}")
    return epoch_loss

In [ ]:
# Training and Validation model.
## Save the model
for epoch in range(EPOCHS):
    train(epoch)
    epoch_loss = valid(epoch,model, valid_loader)
    #if epoch <= 6:
    torch.save(model,f"/content/drive/MyDrive/task-1-semeval/scibert-classify-mod-T1-{epoch}.pt")

# Evaluation Test

In [1]:
!pip install -q transformers
import transformers
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
import os
import torch
import numpy as np
import time
import random
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

     |████████████████████████████████| 1.8MB 19.4MB/s 
     |████████████████████████████████| 890kB 24.3MB/s 
     |████████████████████████████████| 2.9MB 23.4MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Create a SciBERT + Bi-LSTM model for binary classification
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        #self.lstm_hidden_size = lstm_hidden_size
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800, 400)
        self.l4 = torch.nn.Linear(400,100) # check with layer with 30
        self.l5 = torch.nn.Linear(100,2)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        #bert_hidden_layer = bert_hidden_layer.permute(1, 0, 2)   #permute rotates the tensor. if tensor.shape = 3,4,5  tensor.permute(1,0,2), then tensor,shape= 4,3,5  (batch_size, sequence_length, hidden_size)
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        #output_3 = self.l2(output_2)
        output_4 = self.l4(output_2)
        #output_4 = torch.nn.ReLU()(output_4)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)

model = torch.load("/content/drive/MyDrive/task-1-semeval/scibert-classify-mod-T1-1.pt")
model.eval()

SCIBERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [4]:
%rm -r "/content/drive/MyDrive/submission8/"
%mkdir "/content/drive/MyDrive/submission8/"
output_dir = "/content/drive/MyDrive/submission8/"
MAX_LEN=100

In [5]:
%rm -r "/content/drive/MyDrive/submission8/constituency_parsing"
%rm -r "/content/drive/MyDrive/submission8/coreference_resolution"
%rm -r "/content/drive/MyDrive/submission8/data-to-text_generation"
%rm -r "/content/drive/MyDrive/submission8/dependency_parsing"
%rm -r "/content/drive/MyDrive/submission8/document_classification"
%rm -r "/content/drive/MyDrive/submission8/entity_linking"
%rm -r "/content/drive/MyDrive/submission8/face_alignment"
%rm -r "/content/drive/MyDrive/submission8/face_detection"
%rm -r "/content/drive/MyDrive/submission8/hypernym_discovery"
%rm -r "/content/drive/MyDrive/submission8/natural_language_inference"

rm: cannot remove '/content/drive/MyDrive/submission8/constituency_parsing': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/coreference_resolution': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/data-to-text_generation': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/dependency_parsing': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/document_classification': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/entity_linking': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/face_alignment': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/face_detection': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/hypernym_discovery': No such file or directory
rm: cannot remove '/content/drive/MyDrive/submission8/natural_language_inference': No s

In [6]:
%mkdir "/content/drive/MyDrive/submission8/constituency_parsing"
%mkdir "/content/drive/MyDrive/submission8/coreference_resolution"
%mkdir "/content/drive/MyDrive/submission8/data-to-text_generation"
%mkdir "/content/drive/MyDrive/submission8/dependency_parsing"
%mkdir "/content/drive/MyDrive/submission8/document_classification"
%mkdir "/content/drive/MyDrive/submission8/entity_linking"
%mkdir "/content/drive/MyDrive/submission8/face_alignment"
%mkdir "/content/drive/MyDrive/submission8/face_detection"
%mkdir "/content/drive/MyDrive/submission8/hypernym_discovery"
%mkdir "/content/drive/MyDrive/submission8/natural_language_inference"

In [7]:
%rm -r "/content/test"
#!unzip "/content/drive/MyDrive/Sem Eval Task 11 Group 7/Dev v2.zip" -d "/content/test"
!git clone https://github.com/ncg-task/evaluation-phase1.git "/content/test"

rm: cannot remove '/content/test': No such file or directory
Cloning into '/content/test'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (490/490), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 490 (delta 7), reused 488 (delta 5), pack-reused 0
Receiving objects: 100% (490/490), 2.50 MiB | 21.50 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [8]:
#####Testing dataset reading
from shutil import copyfile
#val_input_dir = "/content/drive/My Drive/Dev_v2/"
test_input_dir = "/content/test/"
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_file_name_list = []
test_input_stanza_len = []
for fls in test_list_of_folders:
  count=0
  for i in os.listdir(test_input_dir + fls + '/'):
    count=count+1
    output_info_folder = os.path.join(output_dir,fls,str(i))
    os.makedirs(output_info_folder)
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Grobid-out.txt"):
        copyfile(test_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        #if not os.path.exists(output_info_folder):
        copyfile(test_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
        #stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_len.append(len(stanza_lines_list))
        test_input_stanza_list.append(stanza_lines_list)
      # if files.endswith("sentences.txt"):
      #   sentence_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
      #   sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
      #   test_input_sent_num_list.append(sentence_num_list)
    ### Embedd function
    # sentence_num_list = list(set(sentence_num_list))
    # test_sentence_num_list = [int(s) for s in sentence_num_list]
    # test_sentence_label =[0]*len(stanza_lines_list)
    # for t5 in range(len(test_sentence_num_list)):
    #   t6 = test_sentence_num_list[t5] -1
    #   test_sentence_label[t6] = 1
    print("size of file",len(stanza_lines_list),fls+'/'+str(i))
    with torch.no_grad():
      output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
      if stanza_lines_list[1] != 'abstract':
        output_file.write(str(2) +"\n")
      for t8 in range(2,len(stanza_lines_list)):
        if (stanza_lines_list[t8]=='conclusion') or (stanza_lines_list[t8]=='conclusions'):
          break
        test_input = tokenizer.encode_plus(stanza_lines_list[t8], None, add_special_tokens=True, 
                                           max_length= MAX_LEN, pad_to_max_length=True, 
                                           return_token_type_ids=True, truncation=True, 
                                           return_length = True, return_tensors="pt")
        test_ids = test_input['input_ids'].to(device, dtype = torch.long)
        test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
        test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
        lengths = test_input['length']
        #print("shape",test_ids.shape, test_mask.shape, test_token_type_ids.shape)
        outputs = model(test_ids, test_mask, test_token_type_ids, lengths)
        _, output_idx = torch.max(outputs, dim =1)
        t_output_class = output_idx.cpu().detach().numpy().tolist()
        #print("nfslkn",t_output_class[0],test_sentence_label[t8])
        #output_info_folder = os.path.join(output_dir,fls,str(i))
        # if not os.path.exists(output_info_folder):
        #   os.makedirs(output_info_folder)
        #print("hbas",t_output_class[0])
        #for t7 in range(len(t_output_class)):
          #print(infoname_test_to_idx[t_output_class[t7]])
        output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
        if t_output_class[0] == 1:
          #output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
          output_file.write(str(t8+1) +"\n")
    ## test data and label completed
    ####
    test_file_name_list.append(fls + '/' + str(i))
  print("completed",fls,count)

size of file 213 constituency_parsing/5


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


size of file 90 constituency_parsing/6
size of file 135 constituency_parsing/2
size of file 210 constituency_parsing/3
size of file 223 constituency_parsing/0
size of file 204 constituency_parsing/1
size of file 203 constituency_parsing/8
size of file 230 constituency_parsing/7
size of file 144 constituency_parsing/4
completed constituency_parsing 9
size of file 235 coreference_resolution/5
size of file 127 coreference_resolution/6
size of file 150 coreference_resolution/2
size of file 134 coreference_resolution/3
size of file 256 coreference_resolution/0
size of file 147 coreference_resolution/1
size of file 248 coreference_resolution/8
size of file 129 coreference_resolution/7
size of file 118 coreference_resolution/9
size of file 240 coreference_resolution/4
completed coreference_resolution 10
size of file 319 data-to-text_generation/5
size of file 179 data-to-text_generation/6
size of file 191 data-to-text_generation/2
size of file 126 data-to-text_generation/3
size of file 274 dat

In [10]:
import os
#####Test model
#test_input_dir = "/content/drive/My Drive/Dev_v2/"
test_input_dir = "/content/drive/MyDrive/submission6/"
# test_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#          "relation-classification", "text-classification"]
test_list_of_folders = list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_input_sent_num_list = []
test_file_name_list = []
test_input_entity_list = []
c=0
for fls in test_list_of_folders:
  c1=c1=0
  for i in os.listdir(test_input_dir + fls + '/'):
    c1=c1+1
    c=c+1
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_list.append(stanza_lines_list)
    test_file_name_list.append(fls+'/'+str(i))
  print('completed',c1,fls)
print('evaluation',c)

completed 9 constituency_parsing
completed 10 coreference_resolution
completed 7 data-to-text_generation
completed 9 dependency_parsing
completed 21 document_classification
completed 17 entity_linking
completed 19 face_alignment
completed 22 face_detection
completed 9 hypernym_discovery
completed 32 natural_language_inference
evaluation 155


#Evaluation Script for Sentences

This is the script provided by the organizers for contribution sentences evaluation. The first argument is the path to the folder containing true labels, second argument is the path to the folder containing predicted labels,
third argument is the path to the folder where the "scores.txt" will be stored.
###**IMPORTANT**

Please download the evaluation_for_sentence.py file and place it in /content folder and direct it appropriately.

In [ ]:
!python3 /content/drive/MyDrive/CS779_All_Subtasks/evaluation_for_sentence.py "/content/drive/MyDrive/test" "/content/drive/MyDrive/result_cs779" "/content"

SENT_f1:0.4393564356435644

SENT_precision:0.44823232323232326

SENT_recall:0.4308252427184466
